# Microsoft Agent Framework Sequential Workflow

This notebook demonstrates a fraud detection workflow using the Microsoft Agent Framework with proper data flow patterns. The workflow orchestrates two AI agents to analyze customer data and assess transaction risk using real data from Cosmos DB.

## Overview
- **Customer Data Executor**: Retrieves transaction and customer data from Cosmos DB
- **Risk Analyzer Executor**: Uses Azure AI agent to perform regulatory compliance assessment
- **Data Flow**: Uses `ctx.send_message()` and `ctx.yield_output()` for proper framework integration

## 1. Import Dependencies and Load Environment

Import all necessary libraries for the Microsoft Agent Framework workflow:
- **Agent Framework**: Core workflow components and decorators
- **Azure Integration**: AI agent client and authentication 
- **Database**: Cosmos DB client for data retrieval
- **Data Models**: Pydantic for type-safe data contracts

In [1]:
import asyncio
import os
from typing_extensions import Never
from agent_framework import WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor, ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from azure.cosmos import CosmosClient
from dotenv import load_dotenv
from pydantic import BaseModel

# Load environment variables
load_dotenv(override=True)

True

## 2. Initialize Cosmos DB Connection

Set up connection to Azure Cosmos DB to retrieve real transaction and customer data:
- **Database**: `FinancialComplianceDB` 
- **Collections**: `Customers` and `Transactions`
- **Configuration**: Uses environment variables for endpoint and key

In [2]:
# Initialize Cosmos DB connection
cosmos_endpoint = os.environ.get("COSMOS_ENDPOINT")
cosmos_key = os.environ.get("COSMOS_KEY")
cosmos_client = CosmosClient(cosmos_endpoint, cosmos_key)
database = cosmos_client.get_database_client("FinancialComplianceDB")
customers_container = database.get_container_client("Customers")
transactions_container = database.get_container_client("Transactions")

## 3. Cosmos DB Helper Functions

Define functions to query and retrieve data from Cosmos DB:
- **`get_transaction_data()`**: Retrieves specific transaction by ID
- **`get_customer_data()`**: Retrieves customer profile information  
- **`get_customer_transactions()`**: Gets transaction history for a customer

These functions handle database queries and error cases gracefully.

In [3]:
def get_transaction_data(transaction_id: str) -> dict:
    """Get transaction data from Cosmos DB"""
    try:
        query = f"SELECT * FROM c WHERE c.transaction_id = '{transaction_id}'"
        items = list(transactions_container.query_items(
            query=query,
            enable_cross_partition_query=True
        ))
        return items[0] if items else {"error": f"Transaction {transaction_id} not found"}
    except Exception as e:
        return {"error": str(e)}

def get_customer_data(customer_id: str) -> dict:
    """Get customer data from Cosmos DB"""
    try:
        query = f"SELECT * FROM c WHERE c.customer_id = '{customer_id}'"
        items = list(customers_container.query_items(
            query=query,
            enable_cross_partition_query=True
        ))
        return items[0] if items else {"error": f"Customer {customer_id} not found"}
    except Exception as e:
        return {"error": str(e)}

def get_customer_transactions(customer_id: str) -> list:
    """Get all transactions for a customer from Cosmos DB"""
    try:
        query = f"SELECT * FROM c WHERE c.customer_id = '{customer_id}'"
        items = list(transactions_container.query_items(
            query=query,
            enable_cross_partition_query=True
        ))
        return items
    except Exception as e:
        return [{"error": str(e)}]


## 4. Data Models (Pydantic)

Define type-safe data contracts for the workflow using Pydantic BaseModel:

- **`AnalysisRequest`**: Input data for starting the workflow
- **`CustomerDataResponse`**: Output from customer data executor (includes raw data and analysis)
- **`RiskAnalysisResponse`**: Final output with risk assessment and recommendations

These models ensure data validation and provide clear contracts between executors.

In [4]:
# Request/Response models
class AnalysisRequest(BaseModel):
    message: str
    transaction_id: str = "TX2002"

class CustomerDataResponse(BaseModel):
    customer_data: str
    transaction_data: str
    transaction_id: str
    status: str
    raw_transaction: dict = {}
    raw_customer: dict = {}
    transaction_history: list = []

class RiskAnalysisResponse(BaseModel):
    risk_analysis: str
    risk_score: str
    transaction_id: str
    status: str
    risk_factors: list = []
    recommendation: str = ""
    compliance_notes: str = ""

class ComplianceAuditResponse(BaseModel):
    audit_report_id: str
    audit_conclusion: str
    compliance_rating: str
    risk_factors_identified: list = []
    compliance_concerns: list = []
    recommendations: list = []
    requires_immediate_action: bool = False
    requires_regulatory_filing: bool = False
    transaction_id: str
    status: str

## 5. Customer Data Executor (Intermediate)

The first executor in the workflow chain that:

1. **Retrieves Data**: Gets transaction and customer data from Cosmos DB
2. **Performs Analysis**: Creates comprehensive fraud risk analysis text
3. **Identifies Risk Factors**: Checks for high amounts, risky countries, new accounts, etc.
4. **Sends Data**: Uses `await ctx.send_message(result)` to pass data to next executor

**Key Pattern**: This is an intermediate executor that returns `None` but sends data via the workflow context.

In [5]:
@executor
async def customer_data_executor(
    request: AnalysisRequest,
    ctx: WorkflowContext[CustomerDataResponse]
) -> None:
    """Customer Data Executor that retrieves data from Cosmos DB and sends to next executor."""
    
    try:
        # Get real data from Cosmos DB
        transaction_data = get_transaction_data(request.transaction_id)
        
        if "error" in transaction_data:
            result = CustomerDataResponse(
                customer_data=f"Error: {transaction_data}",
                transaction_data="Error in Cosmos DB retrieval",
                transaction_id=request.transaction_id,
                status="ERROR"
            )
        else:
            customer_id = transaction_data.get("customer_id")
            customer_data = get_customer_data(customer_id)
            transaction_history = get_customer_transactions(customer_id)
            
            # Create comprehensive analysis
            analysis_text = f"""
COSMOS DB DATA ANALYSIS:

Transaction {request.transaction_id}:
- Amount: ${transaction_data.get('amount')} {transaction_data.get('currency')}
- Customer: {customer_id}
- Destination: {transaction_data.get('destination_country')}
- Timestamp: {transaction_data.get('timestamp')}

Customer Profile ({customer_id}):
- Name: {customer_data.get('name')}
- Country: {customer_data.get('country')}
- Account Age: {customer_data.get('account_age_days')} days
- Device Trust Score: {customer_data.get('device_trust_score')}
- Past Fraud: {customer_data.get('past_fraud')}

Transaction History:
- Total Transactions: {len(transaction_history) if isinstance(transaction_history, list) else 0}

FRAUD RISK INDICATORS:
- High Amount: {transaction_data.get('amount', 0) > 10000}
- High Risk Country: {transaction_data.get('destination_country') in ['IR', 'RU', 'NG', 'KP']}
- New Account: {customer_data.get('account_age_days', 0) < 30}
- Low Device Trust: {customer_data.get('device_trust_score', 1.0) < 0.5}
- Past Fraud History: {customer_data.get('past_fraud', False)}

Ready for risk assessment analysis.
"""
            
            result = CustomerDataResponse(
                customer_data=analysis_text,
                transaction_data=f"Workflow analysis for {request.transaction_id}",
                transaction_id=request.transaction_id,
                status="SUCCESS",
                raw_transaction=transaction_data,
                raw_customer=customer_data,
                transaction_history=transaction_history if isinstance(transaction_history, list) else []
            )
        
        # Send data to next executor
        await ctx.send_message(result)
        
    except Exception as e:
        error_result = CustomerDataResponse(
            customer_data=f"Error retrieving data: {str(e)}",
            transaction_data="Error occurred during data retrieval",
            transaction_id=request.transaction_id,
            status="ERROR"
        )
        await ctx.send_message(error_result)


## 6. Risk Analyzer Executor (Intermediate)

The second executor in the workflow chain that:

1. **Receives Data**: Gets `CustomerDataResponse` from previous executor
2. **Connects to AI Agent**: Uses Azure AI agent for regulatory assessment
3. **Performs AI Analysis**: Sends structured prompt to risk analysis agent
4. **Parses Results**: Extracts recommendations and compliance notes
5. **Sends Data**: Uses `await ctx.send_message(result)` to pass data to compliance executor

**Key Pattern**: This is now an intermediate executor that sends data to the compliance report executor.

In [6]:
@executor
async def risk_analyzer_executor(
    customer_response: CustomerDataResponse,
    ctx: WorkflowContext[RiskAnalysisResponse]  # Changed: No longer terminal, sends to next executor
) -> None:
    """Risk Analyzer Executor that processes customer data and sends to compliance executor."""
    
    try:
        # Configuration
        project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
        model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
        RISK_ANALYSER_AGENT_ID = os.getenv("RISK_ANALYSER_AGENT_ID")
        
        if not RISK_ANALYSER_AGENT_ID:
            raise ValueError("RISK_ANALYSER_AGENT_ID required")
        
        async with AzureCliCredential() as credential:
            risk_client = AzureAIAgentClient(
                project_endpoint=project_endpoint,
                model_deployment_name=model_deployment_name,
                async_credential=credential,
                agent_id=RISK_ANALYSER_AGENT_ID
            )
            
            async with risk_client as client:
                risk_agent = ChatAgent(
                    chat_client=client,
                    model_id=model_deployment_name,
                    store=True
                )
                
                # Create risk assessment prompt
                risk_prompt = f"""
Based on the comprehensive fraud analysis provided below, please provide your expert regulatory and compliance risk assessment:

Analysis Data: {customer_response.customer_data}

Please focus on:
1. Validating the risk factors identified in the analysis
2. Assessing the risk score and level from a regulatory perspective
3. Providing additional AML/KYC compliance considerations
4. Checking against sanctions lists and regulatory requirements
5. Final recommendation on transaction approval/blocking/investigation
6. Regulatory reporting requirements if any

Transaction ID: {customer_response.transaction_id}

Provide a structured risk assessment with clear regulatory justification.
"""
                
                result = await risk_agent.run(risk_prompt)
                result_text = result.text if result and hasattr(result, 'text') else "No response from risk agent"
                
                # Parse structured risk data
                risk_factors = []
                recommendation = "INVESTIGATE"  # Default
                compliance_notes = ""
                
                if "HIGH RISK" in result_text.upper() or "BLOCK" in result_text.upper():
                    recommendation = "BLOCK"
                    risk_factors.append("High risk transaction identified")
                elif "LOW RISK" in result_text.upper() or "APPROVE" in result_text.upper():
                    recommendation = "APPROVE"
                
                if "IRAN" in result_text.upper() or "SANCTIONS" in result_text.upper():
                    compliance_notes = "Sanctions compliance review required"
                    
                final_result = RiskAnalysisResponse(
                    risk_analysis=result_text,
                    risk_score="Assessed by Risk Agent based on Cosmos DB data",
                    transaction_id=customer_response.transaction_id,
                    status="SUCCESS",
                    risk_factors=risk_factors,
                    recommendation=recommendation,
                    compliance_notes=compliance_notes
                )
                
                # Send data to next executor (compliance report executor)
                await ctx.send_message(final_result)
        
    except Exception as e:
        error_result = RiskAnalysisResponse(
            risk_analysis=f"Error in risk analysis: {str(e)}",
            risk_score="Unknown",
            transaction_id=customer_response.transaction_id if customer_response else "Unknown",
            status="ERROR"
        )
        await ctx.send_message(error_result)

## 7. Compliance Report Functions

Helper functions for generating audit reports based on risk analysis results:
- **`parse_risk_analysis_result()`**: Extracts key audit information from risk analysis text
- **`generate_audit_report()`**: Creates formal audit reports with compliance ratings
- **`generate_executive_summary()`**: Creates executive-level audit summaries

These functions process the risk analysis output to create structured compliance documentation.

In [7]:
import json
import re
from datetime import datetime
from collections import Counter

def parse_risk_analysis_result(risk_analysis_text: str) -> dict:
    """Parses risk analyser output to extract key audit information."""
    try:
        analysis_data = {
            "original_analysis": risk_analysis_text,
            "parsed_elements": {},
            "audit_findings": []
        }
        
        text_lower = risk_analysis_text.lower()
        
        # Extract risk score
        risk_score_pattern = r'risk\s*score[:\s]*(\d+(?:\.\d+)?)'
        score_match = re.search(risk_score_pattern, text_lower)
        if score_match:
            analysis_data["parsed_elements"]["risk_score"] = float(score_match.group(1))
        
        # Extract risk level
        risk_level_pattern = r'risk\s*level[:\s]*(\w+)'
        level_match = re.search(risk_level_pattern, text_lower)
        if level_match:
            analysis_data["parsed_elements"]["risk_level"] = level_match.group(1).upper()
        
        # Extract transaction ID
        tx_pattern = r'transaction[:\s]*([A-Z0-9]+)'
        tx_match = re.search(tx_pattern, risk_analysis_text)
        if tx_match:
            analysis_data["parsed_elements"]["transaction_id"] = tx_match.group(1)
        
        # Extract key risk factors mentioned
        risk_factors = []
        if "high-risk country" in text_lower or "high risk country" in text_lower:
            risk_factors.append("HIGH_RISK_JURISDICTION")
        if "large amount" in text_lower or "high amount" in text_lower:
            risk_factors.append("UNUSUAL_AMOUNT")
        if "suspicious" in text_lower:
            risk_factors.append("SUSPICIOUS_PATTERN")
        if "sanction" in text_lower:
            risk_factors.append("SANCTIONS_CONCERN")
        if "frequent" in text_lower or "unusual frequency" in text_lower:
            risk_factors.append("FREQUENCY_ANOMALY")
        
        analysis_data["parsed_elements"]["risk_factors"] = risk_factors
        return analysis_data
        
    except Exception as e:
        return {"error": f"Failed to parse risk analysis: {str(e)}"}

def generate_audit_report_from_risk_analysis(risk_analysis_text: str, report_type: str = "TRANSACTION_AUDIT") -> dict:
    """Generates a formal audit report based on risk analyser findings."""
    try:
        parsed_analysis = parse_risk_analysis_result(risk_analysis_text)
        
        if "error" in parsed_analysis:
            return parsed_analysis
        
        elements = parsed_analysis["parsed_elements"]
        
        audit_report = {
            "audit_report_id": f"AUDIT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            "report_type": report_type,
            "generated_timestamp": datetime.now().isoformat(),
            "auditor": "Compliance Report Agent",
            "source_analysis": "Risk Analyser Agent",
            
            "executive_summary": {
                "transaction_id": elements.get("transaction_id", "N/A"),
                "risk_score": elements.get("risk_score", "Not specified"),
                "risk_level": elements.get("risk_level", "Not specified"),
                "audit_conclusion": ""
            },
            
            "detailed_findings": {
                "risk_factors_identified": elements.get("risk_factors", []),
                "compliance_concerns": [],
                "regulatory_implications": [],
                "recommendations": []
            },
            
            "compliance_status": {
                "requires_regulatory_filing": False,
                "requires_enhanced_monitoring": False,
                "requires_immediate_action": False,
                "compliance_rating": "PENDING"
            }
        }
        
        # Analyze risk score for audit conclusions
        risk_score = elements.get("risk_score", 0)
        if isinstance(risk_score, (int, float)):
            if risk_score >= 80:
                audit_report["executive_summary"]["audit_conclusion"] = "HIGH RISK - Immediate review required"
                audit_report["compliance_status"]["requires_immediate_action"] = True
                audit_report["compliance_status"]["compliance_rating"] = "NON_COMPLIANT"
            elif risk_score >= 50:
                audit_report["executive_summary"]["audit_conclusion"] = "MEDIUM RISK - Enhanced monitoring recommended"
                audit_report["compliance_status"]["requires_enhanced_monitoring"] = True
                audit_report["compliance_status"]["compliance_rating"] = "CONDITIONAL_COMPLIANCE"
            else:
                audit_report["executive_summary"]["audit_conclusion"] = "LOW RISK - Standard monitoring sufficient"
                audit_report["compliance_status"]["compliance_rating"] = "COMPLIANT"
        
        # Add specific findings based on risk factors
        risk_factors = elements.get("risk_factors", [])
        
        if "HIGH_RISK_JURISDICTION" in risk_factors:
            audit_report["detailed_findings"]["compliance_concerns"].append(
                "Transaction involves high-risk jurisdiction requiring enhanced monitoring"
            )
            audit_report["compliance_status"]["requires_regulatory_filing"] = True
        
        if "SANCTIONS_CONCERN" in risk_factors:
            audit_report["detailed_findings"]["compliance_concerns"].append(
                "Potential sanctions-related issues identified in risk analysis"
            )
            audit_report["compliance_status"]["requires_immediate_action"] = True
        
        # Generate recommendations
        if audit_report["compliance_status"]["requires_immediate_action"]:
            audit_report["detailed_findings"]["recommendations"].extend([
                "Freeze transaction pending investigation",
                "Conduct enhanced customer due diligence",
                "File suspicious activity report with regulators"
            ])
        elif audit_report["compliance_status"]["requires_enhanced_monitoring"]:
            audit_report["detailed_findings"]["recommendations"].extend([
                "Place customer on enhanced monitoring list",
                "Review transaction against internal risk policies"
            ])
        else:
            audit_report["detailed_findings"]["recommendations"].append(
                "Continue standard monitoring procedures"
            )
        
        return audit_report
        
    except Exception as e:
        return {"error": f"Failed to generate audit report: {str(e)}"}


## 8. Compliance Report Executor (Terminal)

The third executor in the workflow chain that processes risk analysis results:

1. **Receives Risk Analysis**: Gets `RiskAnalysisResponse` from risk analyzer executor
2. **Connects to AI Agent**: Uses Azure AI compliance report agent for audit report generation
3. **Generates Audit Report**: Creates formal compliance documentation
4. **Parses Audit Results**: Extracts compliance ratings and recommendations
5. **Yields Final Output**: Uses `await ctx.yield_output(result)` to provide final audit report

**Key Pattern**: This executor replaces the risk analyzer as the terminal executor, extending the workflow chain.

In [8]:
@executor
async def compliance_report_executor(
    risk_response: RiskAnalysisResponse,
    ctx: WorkflowContext[Never, ComplianceAuditResponse]
) -> None:
    """Compliance Report Executor that generates audit reports from risk analysis results."""
    
    try:
        # Configuration
        project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
        model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
        COMPLIANCE_REPORT_AGENT_ID = os.getenv("COMPLIANCE_REPORT_AGENT_ID")
        
        # If no specific compliance agent, we can generate the report locally
        if not COMPLIANCE_REPORT_AGENT_ID:
            # Generate audit report using local functions
            audit_report = generate_audit_report_from_risk_analysis(
                risk_analysis_text=risk_response.risk_analysis,
                report_type="TRANSACTION_AUDIT"
            )
            
            if "error" in audit_report:
                error_result = ComplianceAuditResponse(
                    audit_report_id="ERROR_REPORT",
                    audit_conclusion=f"Error generating audit report: {audit_report['error']}",
                    compliance_rating="ERROR",
                    transaction_id=risk_response.transaction_id,
                    status="ERROR"
                )
                await ctx.yield_output(error_result)
                return
            
            # Convert audit report to response model
            final_result = ComplianceAuditResponse(
                audit_report_id=audit_report["audit_report_id"],
                audit_conclusion=audit_report["executive_summary"]["audit_conclusion"],
                compliance_rating=audit_report["compliance_status"]["compliance_rating"],
                risk_factors_identified=audit_report["detailed_findings"]["risk_factors_identified"],
                compliance_concerns=audit_report["detailed_findings"]["compliance_concerns"],
                recommendations=audit_report["detailed_findings"]["recommendations"],
                requires_immediate_action=audit_report["compliance_status"]["requires_immediate_action"],
                requires_regulatory_filing=audit_report["compliance_status"]["requires_regulatory_filing"],
                transaction_id=risk_response.transaction_id,
                status="SUCCESS"
            )
            
            await ctx.yield_output(final_result)
            return
        
        # Use Azure AI agent for compliance reporting
        async with AzureCliCredential() as credential:
            compliance_client = AzureAIAgentClient(
                project_endpoint=project_endpoint,
                model_deployment_name=model_deployment_name,
                async_credential=credential,
                agent_id=COMPLIANCE_REPORT_AGENT_ID
            )
            
            async with compliance_client as client:
                compliance_agent = ChatAgent(
                    chat_client=client,
                    model_id=model_deployment_name,
                    store=True
                )
                
                # Create compliance report prompt
                compliance_prompt = f"""
Based on the following Risk Analyser Agent output, please generate a comprehensive audit report:

Risk Analysis Result:
{risk_response.risk_analysis}

Transaction ID: {risk_response.transaction_id}
Risk Score: {risk_response.risk_score}
Recommendation: {risk_response.recommendation}
Risk Factors: {risk_response.risk_factors}
Compliance Notes: {risk_response.compliance_notes}

Please provide:
1. Formal audit report with compliance ratings based on the risk analysis
2. Specific required actions and recommendations derived from the findings
3. Executive summary of key audit conclusions
4. Compliance status and regulatory requirements

Focus on translating the risk analysis into clear audit findings and actionable recommendations for management review.
"""
                
                result = await compliance_agent.run(compliance_prompt)
                result_text = result.text if result and hasattr(result, 'text') else "No response from compliance agent"
                
                # Generate structured audit report locally and combine with AI response
                local_audit = generate_audit_report_from_risk_analysis(risk_response.risk_analysis)
                
                if "error" not in local_audit:
                    final_result = ComplianceAuditResponse(
                        audit_report_id=local_audit["audit_report_id"],
                        audit_conclusion=f"{local_audit['executive_summary']['audit_conclusion']} (AI Enhanced: {result_text[:200]}...)",
                        compliance_rating=local_audit["compliance_status"]["compliance_rating"],
                        risk_factors_identified=local_audit["detailed_findings"]["risk_factors_identified"],
                        compliance_concerns=local_audit["detailed_findings"]["compliance_concerns"],
                        recommendations=local_audit["detailed_findings"]["recommendations"],
                        requires_immediate_action=local_audit["compliance_status"]["requires_immediate_action"],
                        requires_regulatory_filing=local_audit["compliance_status"]["requires_regulatory_filing"],
                        transaction_id=risk_response.transaction_id,
                        status="SUCCESS"
                    )
                else:
                    # Fallback if local audit fails
                    final_result = ComplianceAuditResponse(
                        audit_report_id=f"AI_AUDIT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                        audit_conclusion=result_text[:500] if len(result_text) > 500 else result_text,
                        compliance_rating="AI_GENERATED",
                        transaction_id=risk_response.transaction_id,
                        status="SUCCESS"
                    )
                
                await ctx.yield_output(final_result)
        
    except Exception as e:
        error_result = ComplianceAuditResponse(
            audit_report_id="ERROR_REPORT",
            audit_conclusion=f"Error in compliance reporting: {str(e)}",
            compliance_rating="ERROR",
            transaction_id=risk_response.transaction_id if risk_response else "Unknown",
            status="ERROR"
        )
        await ctx.yield_output(error_result)

## 9. Workflow Construction and Execution

Build and execute the three-executor workflow using Microsoft Agent Framework:

1. **Build Workflow**: Uses `WorkflowBuilder` to connect all three executors with edges
2. **Workflow Chain**: Customer Data → Risk Analyzer → Compliance Report
3. **Create Request**: Defines input data for the workflow
4. **Stream Execution**: Uses `workflow.run_stream()` to capture events in real-time
5. **Capture Output**: Listens for `WorkflowOutputEvent` to get final compliance audit results

**Key Pattern**: The workflow now has three sequential executors with proper data flow between each stage.

In [9]:
async def run_fraud_detection_workflow():
    """Execute the fraud detection workflow using Microsoft Agent Framework."""
    
    # Build workflow with three executors
    workflow = (
        WorkflowBuilder()
        .set_start_executor(customer_data_executor)
        .add_edge(customer_data_executor, risk_analyzer_executor)
        .add_edge(risk_analyzer_executor, compliance_report_executor)  # New edge
        .build()
    )
    
    # Create request
    request = AnalysisRequest(
        message="Comprehensive fraud analysis",
        transaction_id="TX2002"
    )
    
    # Execute workflow with streaming
    final_output = None
    
    async for event in workflow.run_stream(request):
        # Capture final workflow output
        if isinstance(event, WorkflowOutputEvent):
            final_output = event.data
    
    return final_output

## 10. Main Function and Result Display

The main function that orchestrates the workflow execution:

1. **Runs Workflow**: Calls the three-stage fraud detection workflow
2. **Handles Results**: Processes the final `ComplianceAuditResponse`
3. **Displays Output**: Shows comprehensive audit information including:
   - Audit report ID and compliance rating
   - Risk factors and compliance concerns
   - Specific recommendations and required actions
   - Regulatory filing requirements
4. **Error Handling**: Catches and reports any workflow execution errors

This provides a complete audit trail from data analysis through compliance reporting.

In [10]:
async def main():
    """Main function to run the fraud detection workflow."""
    try:
        result = await run_fraud_detection_workflow()
        
        # Display results - now expects ComplianceAuditResponse
        if result and isinstance(result, ComplianceAuditResponse):
            print(f"Audit Report ID: {result.audit_report_id}")
            print(f"Transaction: {result.transaction_id}")
            print(f"Status: {result.status}")
            print(f"Compliance Rating: {result.compliance_rating}")
            print(f"Audit Conclusion: {result.audit_conclusion}")
            
            if result.risk_factors_identified:
                print(f"Risk Factors: {result.risk_factors_identified}")
            if result.compliance_concerns:
                print(f"Compliance Concerns: {result.compliance_concerns}")
            if result.recommendations:
                print(f"Recommendations: {result.recommendations}")
            if result.requires_immediate_action:
                print("⚠️  IMMEDIATE ACTION REQUIRED")
            if result.requires_regulatory_filing:
                print("📋 REGULATORY FILING REQUIRED")
        
        return result
        
    except Exception as e:
        print(f"Workflow execution failed: {str(e)}")
        return None

## 11. Execute the Complete Workflow

Run the enhanced fraud detection workflow with compliance reporting.

**Note**: In Jupyter notebooks, we use `await` directly instead of `asyncio.run()` because the notebook already runs in an asyncio event loop.

The complete workflow will:
1. **Customer Data Analysis**: Retrieve transaction TX1001 from Cosmos DB and analyze risk factors
2. **Risk Assessment**: Use Azure AI agent for regulatory compliance assessment  
3. **Compliance Reporting**: Generate formal audit report with compliance ratings and recommendations
4. **Return Structured Results**: Provide comprehensive audit documentation with actionable insights

This demonstrates a complete end-to-end compliance workflow using the Microsoft Agent Framework.

In [11]:
result = await main()

Audit Report ID: AUDIT_20251210_120213
Transaction: TX2002
Status: SUCCESS
Compliance Rating: COMPLIANT
Audit Conclusion: LOW RISK - Standard monitoring sufficient (AI Enhanced: **Compliance Audit Report**

---

**Audit Report for Transaction ID:** TX2002  
**Date:** [Insert Date of Report]  
**Auditor:** Compliance Audit Report Agent

---

### 1. Audit Overview and Objective...)
Risk Factors: ['HIGH_RISK_JURISDICTION', 'UNUSUAL_AMOUNT', 'SUSPICIOUS_PATTERN', 'SANCTIONS_CONCERN']
Compliance Concerns: ['Transaction involves high-risk jurisdiction requiring enhanced monitoring', 'Potential sanctions-related issues identified in risk analysis']
Recommendations: ['Freeze transaction pending investigation', 'Conduct enhanced customer due diligence', 'File suspicious activity report with regulators']
⚠️  IMMEDIATE ACTION REQUIRED
📋 REGULATORY FILING REQUIRED
